In [1]:
#%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 --force-reinstall

In [2]:
import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric

Installing packages for torch-2.5.1+cu124...
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install scikit-learn imbalanced-learn xgboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install optuna

Note: you may need to restart the kernel to use updated packages.


### Hyperparameter Optimization Citeseer
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 7),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [1,5,10,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "citeseer_scar_sampling_nnif_batch_cluster.csv",
        "min":0.835,
        "clusters":trial.suggest_categorical("clusters", [100,200,300,400,500]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-12 11:20:21,411] A new study created in memory with name: no-name-c7765fdb-3828-49f5-bbd8-fe3279a0a332


Running experiment with seed=654:
 - K=4, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=15, reliable_mini_batch=True
 - ratio=0.09767575025383361, aggregation=sum, treatment=relabeling
 - model_type=SAGEConv, rate_pairs=10, clusters=100, lr=0.00901690252087701


Computing METIS partitioning...
Done!


Epoch 0, Loss: 4.0619
Epoch 10, Loss: 3.1184
Epoch 20, Loss: 2.0205
Epoch 30, Loss: 1.3516
Epoch 40, Loss: 0.9404


[I 2025-03-12 11:20:52,031] Trial 0 finished with value: 0.676602086438152 and parameters: {'K': 4, 'layers': 2, 'hidden_channels': 256, 'out_channels': 64, 'ratio': 0.09767575025383361, 'aggregation': 'sum', 'treatment': 'relabeling', 'model_type': 'SAGEConv', 'rate_pairs': 10, 'batch_size': 15, 'lr': 0.00901690252087701, 'clusters': 100}. Best is trial 0 with value: 0.676602086438152.


 - Metrics: Accuracy=0.8696, F1=0.6766, Recall=0.6476, Precision=0.7083
F1 = 0.68 < 0.835, skipping ...
Done. Results written to citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_1203112021.csv.
Average F1 over valid seeds: 0.6766 ± 0.0000
Running experiment with seed=654:
 - K=4, layers=1, hidden=64, out=64
 - norm=None, dropout=0, batch_size=1, reliable_mini_batch=True
 - ratio=0.10150501612558517, aggregation=sum, treatment=removal
 - model_type=GINConv, rate_pairs=9, clusters=100, lr=0.007912856249357784


Computing METIS partitioning...
Done!
[I 2025-03-12 11:20:55,441] Trial 1 finished with value: 0.0 and parameters: {'K': 4, 'layers': 1, 'hidden_channels': 64, 'out_channels': 64, 'ratio': 0.10150501612558517, 'aggregation': 'sum', 'treatment': 'removal', 'model_type': 'GINConv', 'rate_pairs': 9, 'batch_size': 1, 'lr': 0.007912856249357784, 'clusters': 100}. Best is trial 0 with value: 0.676602086438152.


Error: Labels must be in {0,1}, got [0].
Done. Results written to citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_1203112052.csv.
Average F1 over valid seeds: 0.0000 ± 0.0000
Running experiment with seed=654:
 - K=3, layers=1, hidden=256, out=64
 - norm=None, dropout=0, batch_size=5, reliable_mini_batch=True
 - ratio=0.09189430602320858, aggregation=sum, treatment=removal
 - model_type=GATConv, rate_pairs=1, clusters=100, lr=0.004221047121711186


Computing METIS partitioning...
Done!


Epoch 0, Loss: 14.4128
Epoch 10, Loss: 8.5733
Epoch 20, Loss: 4.9554
Epoch 30, Loss: 3.1662


### Hyperparameter Optimization Citeseer
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 7),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [128,256,512,1024,2048]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "citeseer_sar_sampling_nnif_batch_cluster.csv",
        "min":0.82
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Cora
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 4, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [1,5,10,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "cora_scar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":trial.suggest_categorical("clusters", [100,200,300,400,500]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Cora
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 4, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [128,256,512,1024,2048]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "cora_sar_sampling_nnif_batch_cluster.csv",
        "min":0.84
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Pubmed
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 5, 14),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.2, 0.4),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [1,5,10,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "pubmed_scar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":trial.suggest_categorical("clusters", [100,200,300,400,500]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Pubmed
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 6, 14),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.23, 0.27),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [512,1024,2048,5096,10192]),
        "lr": trial.suggest_float("lr", 1e-5, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "pubmed_sar_sampling_nnif_batch_cluster.csv",
        "min": 0.82
        }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization WikiCS
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 3, 40),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [1,5,10,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 5,
        "output_csv": "wikics_scar_sampling_nnif_batch_cluster.csv",
        "min":0.88,
        "clusters":trial.suggest_categorical("clusters", [100,200,300,400,500]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization WikiCS
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 23, 48),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,#trial.suggest_float("dropout", 0, 0.5),
        "margin": 0.5,
        "lpl_weight": 0.5,#trial.suggest_float("lpl_weight", 0.01, 0.99),
        "ratio": trial.suggest_float("ratio", 0.08, 0.48),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv', 'TransformerConv']),
        "sampling_mode":trial.suggest_categorical("sampling_mode", [None,'nn', 'random', 'feature']), #'weighted',
        "num_neighbors": trial.suggest_int("num_neighbors", 5, 20),
        "seeds": 5,
        "output_csv": "wikics_sar_conv_sampling.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


# Elliptic Bitcoin

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "elliptic-bitcoin",      
        "mechanism": "SAR2",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 3, 20),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0, 0.4),
        "ratio": trial.suggest_float("ratio", 0.001, 0.03),
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv', 'SAGEConv', 'GINConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [10,20,30,40]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "reliable_mini_batch":True,
        "seeds": 1,
        "output_csv": "elliptic_bitcoin_cluster.csv",
        "min":0.65,
        "clusters":trial.suggest_categorical("clusters", [1300,1500,1700,2000])
        }

    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-12 11:08:16,457] A new study created in memory with name: no-name-9f843de6-1c81-4682-859c-d54c36ea9fdb


Running experiment with seed=654:
 - K=16, layers=2, hidden=256, out=128
 - norm=None, dropout=0.38706779927156976, batch_size=40, reliable_mini_batch=True
 - ratio=0.01222827988140382, aggregation=sum, treatment=removal
 - model_type=SAGEConv, rate_pairs=3, clusters=2000, lr=0.005651409078608822
165
torch.Size([203769, 165])


Computing METIS partitioning...
Done!


Epoch 0, Loss: 674.1356
Epoch 10, Loss: 127.3564
Epoch 20, Loss: 50.7907
Epoch 30, Loss: 26.9527
Epoch 40, Loss: 16.4045


[I 2025-03-12 11:19:05,538] Trial 0 finished with value: 0.6324221233312143 and parameters: {'K': 16, 'layers': 2, 'hidden_channels': 256, 'out_channels': 128, 'dropout': 0.38706779927156976, 'ratio': 0.01222827988140382, 'aggregation': 'sum', 'treatment': 'removal', 'model_type': 'SAGEConv', 'rate_pairs': 3, 'batch_size': 40, 'lr': 0.005651409078608822, 'clusters': 2000}. Best is trial 0 with value: 0.6324221233312143.


 - Metrics: Accuracy=0.9379, F1=0.6324, Recall=0.5472, Precision=0.7491
F1 = 0.63 < 0.65, skipping ...
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_1203110816.csv.
Average F1 over valid seeds: 0.6324 ± 0.0000
Running experiment with seed=654:
 - K=8, layers=1, hidden=64, out=64
 - norm=None, dropout=0.16686621113334402, batch_size=40, reliable_mini_batch=True
 - ratio=0.01494468463150106, aggregation=sum, treatment=relabeling
 - model_type=SAGEConv, rate_pairs=1, clusters=2000, lr=0.000584308231031967
165
torch.Size([203769, 165])


Computing METIS partitioning...
